In [1]:
#if jupyternotify is installed, we can add %notify to a cell to get an alert when it ifnished running
%load_ext jupyternotify

<IPython.core.display.Javascript object>

In [2]:
import metrics_helpers as indicators
import pickle as pk
import gc
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

# Create Indicators

In [3]:
def dt_to_int(dt): #datetime to integer
    return dt.astype('int')/(10**9)

In [4]:
#netmums


with open('/Users/sma/Documents/INRAE internship/scrape-git/netmums/allposts_rerun.pkl', 'rb') as f:
    netmums = pk.load(f)

    
nm_ind = indicators.indicators(netmums, fb=False)
#this one takes long, around 20 seconds I think.

/usr/local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
import pandas as pd

## Create two term totals
combine counts for terms which are hazards and terms which are products.

In [11]:
#choose terms to count in totals
term_count_df = nmdf[nmdf.columns[pd.Series(nmdf.columns).str.startswith('term_counts_')]]

products = term_count_df.columns[[5,12,13,17,33, 34, 35, 44,45,57, 58, 65,66,68, 69,70,78, 79,80, 85]]
hazards = term_count_df.columns[[1,2,3,9,10,11,14,15,16,18,
                                 19,20,21,22,23,24,25,26,27,
                                 28,30,31,37,38,39,40,
                                 41,42,43,46,47,48,49,50,51,
                                 52,53,54,55,56,59,60,61,62,
                                 63,64,67,71,72,73,74,75,76,
                                 77,81,82,83,84]]


In [12]:
hazards

Index(['term_counts_acrylamid', 'term_counts_acrylamide',
       'term_counts_additives', 'term_counts_bacteria',
       'term_counts_biphenyls', 'term_counts_bisphenol', 'term_counts_bpa',
       'term_counts_campylobacter', 'term_counts_carcinogen',
       'term_counts_chemical', 'term_counts_chemicals',
       'term_counts_contaminants', 'term_counts_contaminated',
       'term_counts_cronobacter', 'term_counts_deoxynivalenol',
       'term_counts_dioxin', 'term_counts_disruptor', 'term_counts_drugs',
       'term_counts_e-coli', 'term_counts_ecoli', 'term_counts_endocrine',
       'term_counts_estrogen', 'term_counts_fungus', 'term_counts_genetically',
       'term_counts_gm', 'term_counts_gmo', 'term_counts_hazard',
       'term_counts_histamine', 'term_counts_hydrocarbons',
       'term_counts_listeria', 'term_counts_metals', 'term_counts_microbes',
       'term_counts_microbiologic', 'term_counts_moah', 'term_counts_modified',
       'term_counts_moh', 'term_counts_mold', 'term_

In [13]:
products

Index(['term_counts_applesauce', 'term_counts_bottle',
       'term_counts_bottle-fed', 'term_counts_cereal', 'term_counts_food',
       'term_counts_for', 'term_counts_formula', 'term_counts_jar',
       'term_counts_jarred', 'term_counts_oatmeal', 'term_counts_oats',
       'term_counts_porridge', 'term_counts_premade', 'term_counts_pudding',
       'term_counts_puree', 'term_counts_purée', 'term_counts_veg',
       'term_counts_vegetable', 'term_counts_veggie', 'term_counts_yoghurt'],
      dtype='object')

create the totals of terms in two categories.

In [14]:
#netmums
#add totals
term_count_df = nmdf[nmdf.columns[pd.Series(nmdf.columns).str.startswith('term_counts_')]]

products = term_count_df.columns[[5,12,13,17,33, 34, 35, 44,45,57, 58, 65,66,68, 69,70,78, 79,80, 85]]
hazards = term_count_df.columns[[1,2,3,4,6,7,8,9,10,11,14,15,16,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,36,37,38,39,40,41,42,43,46,47,48,49,50,51,52,53,54,55,56,59,60,61,62,63,64,67,71,72,73,74,75,76,77,81,82,83,84]]


term_count_hazard_df = term_count_df[hazards]
term_count_product_df = term_count_df[products]

url_term_count_df = nmdf[nmdf.columns[pd.Series(nmdf.columns).str.startswith('url_term_counts_')]]

url_term_count_hazard_df = url_term_count_df['url_' + hazards]
url_term_count_product_df = url_term_count_df['url_' + products] 


nmdf['total_term_count'] = term_count_df.sum(axis=1) #make sure there's no s because we can easily filter the non totals then with contains.
nmdf['total_url_term_count'] = term_count_df.sum(axis=1)

nmdf['total_term_count_hazard'] = term_count_hazard_df.sum(axis=1) #make sure there's no s because we can easily filter the non totals then with contains.
nmdf['total_url_term_count_hazard'] = url_term_count_hazard_df.sum(axis=1)

nmdf['total_term_count_product'] = term_count_product_df.sum(axis=1) #make sure there's no s because we can easily filter the non totals then with contains.
nmdf['total_url_term_count_product'] = url_term_count_product_df.sum(axis=1)

# Make Indicators Relative so More Easily Interpreted.

In [20]:
#TODO adjust this for the new indicators...
#will involve some playing around / investigating I guess.

nmdf = nmdf.copy()
#so we don't get an error for operating on a slice or sth

In [21]:
nmdf['total_term_count'] = nmdf['total_term_count'] / (nmdf['avg_post_length'] + 1)
nmdf['total_term_count_hazard'] = nmdf['total_term_count_hazard'] / (nmdf['avg_post_length'] + 1)
nmdf['total_term_count_product'] = nmdf['total_term_count_product'] / (nmdf['avg_post_length'] + 1)
#FIXME divide by available comments insteand of average post length (for netmums only.)

nmdf['num_unique_posters'] = nmdf['num_unique_posters'] / (nmdf['available_comments'] + 1)

## Modify Time so We can Graph It.

In [ ]:
from datetime import datetime
nmdf['int_post_time'] = dt_to_int(nmdf.post_time.dropna()) 

# Pair Plots

Take a sample of 500 points and plot the pair plots of our indicators.
Datetime could not be plotted, so it's converted to int. We can estimate on the graph what the corresponding date is by looking at quantiles.
Lexical richness uses a measure called MTLD, which should be less biased for short documents than the most common TTR (Type-Token Ratio)


In [23]:
nm_indicators = \
['total_term_count',
'total_term_count_hazard',
'total_term_count_product',
'num_unique_posters',
'avg_post_length',
'lexical_richness', 
'term_distance_simple']



# Final Set of Criterion

 * year >= 2016
 * non-zero occurences of both a product and a hazard term
 * distance between product and hazard is below the 95th percentile.
 
 * if this approach proves not to yield results, we can instead attempt to take a subset based on the subforum which results occur in !

# Basic Descriptive Statistics


## Netmums

### Export the Keys for our Desired Subset.

In [26]:
#export the keys #TODO
temp = nmdf.loc[nmdf.total_term_count_hazard > 0].loc[nmdf.total_term_count_product > 0].loc[nmdf.post_time >= time_cutoff].loc[nmdf.term_distance_simple <= 491]

# Clean Text
We now construct our subset form the desired keys and then process the text.
* remove typos of relevant words using Levenshtein Distances
* replace tokens for specific foods and brands with their category, after compiling lists of these terms using word2vec
    * replace tokens for all types of fruits with fruit
    * replace tokens for all types of vegetable with vegetable
    * replace tokens for all types of grains with "cereal" (???) should I??

#### Initialization 

In [27]:
import fuzzy_typos
import time

In [28]:
foodwords = [
#infant formula
"formula","baby formula", "bottle-fed", "bottle",
#sterizlized vegetable mixed with fish
"veggie baby food","vegetable baby food",
"veg puree", "veg purée",
#fresh fruit puree mildly processed
"fruit puree","fruit baby food", "fruit purée", "applesauce",
#infant cereals
"cereal for baby", "cereal", "porridge", "oats", "oatmeal",
#other
"jar food", "baby food", "jarred", "premade food", "puree", "purée", "jarred food"
,"yoghurt", "pudding"]

In [29]:
hazardwords = ["Chemical contaminants",#ENDOCRINE DISRUPTOR
"Endocrine disruptor","endocrine","estrogen",#end
#FOOD PRESERVATIVES, SWEETENERS AND ADDITIVES
"preservatives","sweeteners","additives", #end
"Pesticides",#VETERINARY DRUGS
"Veterinary drugs","animal drugs","vet drugs", #end
#GMO
"GMO","GM","genetically modified",#end
"Metals","Mycotoxin",#BISPHENOL A
"Bisphenol A","BPA", #end
#FURAN - removed because nothing related to this returns results
#DON (note that this acronym nobody uses and all results are from words like "don't")
"deoxynivalenol","vomitoxin",#end
#DIOXIN AND PCB
"Dioxin","PCB","biphenyls",#end
#MOH
"MOH","hydrocarbons","saturated hydrocarbons","MOAH","aromatic hydrocarbons",#end
"Nitrates",
#ACRYLAMID
"Acrylamid",
"Acrylamide",
"phthalates",
#MICROBIOLOGIC CONTAMINANTS
"Microbiologic contaminants","spores","mold","mould","virus","microbes","contaminated",#end
"Salmonella","Campylobacter","Listeria",
#ECOLI
"EColi",
"E-coli", #end
"Cronobacter",
"Histamine",
#other bacteria
"bacteria",#end
"Virus",
"Parasites",
#UNRELATED BUT MAYBE USEFUL?
"carcinogen","chemicals", "toxic", "toxin", "poisonous", "fungus", "food poisoning", "hazard","EFSA","European Food Safety Authority"]

In [30]:
f = fuzzy_typos.fuzzy_typos(foodwords)
h = fuzzy_typos.fuzzy_typos(hazardwords)

#### Parallelization

In [31]:
from joblib import Parallel, delayed

In [60]:
keys = list(nm_ind.text_dict.keys())
num_keys = len(keys)
num_lists = 20 #how many instances will be split for parallel processing
list_of_list_of_keys = [keys[slice(i,num_keys,num_lists)] for i in range(num_lists)]

def get_small_dict(list_of_keys): #we give process small dicts because o.w. the whole dict (a global) will get duplicated in each instance
    return {key: nm_ind.text_dict[key] for key in list_of_keys}

def process(small_dict):
    return {key: f.fix_typos(value) for key, value in small_dict.items()}

In [61]:
list_of_small_dict = [get_small_dict(i) for i in list_of_list_of_keys]

In [34]:
start = time.time()
results = Parallel(n_jobs=-1)(delayed(process)(i) for i in list_of_small_dict)
end = time.time()
print('default food words time: ' + str(end - start),)
%notify

default food words time: 877.6941518783569


<IPython.core.display.Javascript object>

In [35]:
untypod_dict = {key:value for dictionary in results for key,value in dictionary.items()}

In [38]:
len(untypod_dict.keys())

6523

In [39]:
len(nm_ind.text_dict.keys())

6523

In [47]:
netmums[blah]


{'query': {'https://www.netmums.com/search/chat/Chemical contaminants baby food',
  'https://www.netmums.com/search/chat/Chemical contaminants baby formula',
  'https://www.netmums.com/search/chat/Chemical contaminants bottle',
  'https://www.netmums.com/search/chat/Chemical contaminants bottle-fed',
  'https://www.netmums.com/search/chat/Chemical contaminants cereal for baby',
  'https://www.netmums.com/search/chat/Chemical contaminants formula',
  'https://www.netmums.com/search/chat/Chemical contaminants jar food',
  'https://www.netmums.com/search/chat/Chemical contaminants yoghurt',
  'https://www.netmums.com/search/chat/MOH bottle-fed',
  'https://www.netmums.com/search/chat/Metals baby food',
  'https://www.netmums.com/search/chat/Metals baby formula',
  'https://www.netmums.com/search/chat/Metals bottle',
  'https://www.netmums.com/search/chat/Metals bottle-fed',
  'https://www.netmums.com/search/chat/Metals formula',
  'https://www.netmums.com/search/chat/chemicals baby formul

In [58]:
#TODO: modify process() to work on the relelvant entries in the value of a dict
#TODO: modify get_small_dict to work on the actual dict and not text_dict.

#THE RELEVANT THINGS:
#netmums[blah]['title']
#netmums[blah]['posts'][n]['body']
#netmums[blah]['posts'][n]['quotes_w']
#netmums[blah]['posts'][n]['quotes_y']['text']

def get_small_dict(list_of_keys): #we give process small dicts because o.w. the whole dict (a global) will get duplicated in each instance
    return {key: netmums[key] for key in list_of_keys}

def process(typofixer,small_dict): #now process takes two objects
    #approx 1.5x slower than the text_dict way.
    for key, value in small_dict.items():
        small_dict[key]['title'] = typofixer.fix_typos(small_dict[key]['title'])
        for ind, item in enumerate(small_dict[key]['posts']): #a list of dicts
            if item['body']:
                small_dict[key]['posts'][ind]['body'] = typofixer.fix_typos(item['body'])
            if item['quotes_w']:
                for qind, quote in enumerate(item['quotes_w']):
                    small_dict[key]['posts'][ind]['quotes_w'][qind] = typofixer.fix_typos(quote)
            if item['quotes_y']:
                for qind, quote in enumerate(item['quotes_y']):
                    small_dict[key]['posts'][ind]['quotes_y'][qind]['text'] = typofixer.fix_typos(quote['text'])     
    return small_dict

In [52]:
list_of_small_dict = [get_small_dict(i) for i in list_of_list_of_keys]

In [59]:
start = time.time()
one_fixed_dict = process(f, list_of_small_dict[0])
end = time.time()
print('default food words time: ' + str(end - start),)
%notify

default food words time: 93.44288301467896


<IPython.core.display.Javascript object>

In [ ]:
#TODO: define three different set of values and their corresponding fuzzy_typo objects
#single_token_words_to_replace = {}
#phrases_to_replace {}
#remaining_words = all - single_token_words_to_replace - phrases_to_replace

def process(small_dict):
    temp = {key: typoreplacer.fix_typos(value) for key, value in small_dict.items()}
    temp = {key: h.fix_typos(value) for key, value in small_dict.items()}
    return 

## trying to do threading and the thign agian....
# update: this is useless we dont need multithrading. I THINK? it is only slower or the same but maybe its useful when we adapt it.


In [322]:
len(problematic)

190

In [323]:
len({j:val for i in results for j,val in i.items()})

190

In [58]:
def run_multithread(thing):
    return Parallel(n_jobs=-1, prefer='threads')(delayed(process)(i) for i in thing)

In [68]:
keys = list(nm_ind.text_dict.keys())
num_keys = len(keys)
num_lists = 20
num_lol = 10 #same as number of cores
list_of_list_of_keys = [keys[slice(i,num_keys,num_lists*num_lol)] for i in range(num_lists*num_lol)]
#note that num_keys == len(list_of_list_of_keys) by definition..
list_of_lol_of_keys = [list_of_list_of_keys[slice(i,num_keys,num_lol)] for i in range(num_lol)]

In [69]:
#create dicts
lol_of_small_dicts = [[get_small_dict(keys) for keys in list_of_list] for list_of_list in list_of_lol_of_keys]

In [70]:
start = time.time()
results = Parallel(n_jobs=-1, prefer='processes')(delayed(run_multithread)(i) for i in lol_of_small_dicts)
end = time.time()
print('default food words time: ' + str(end - start),)
%notify

default food words time: 732.8989160060883


<IPython.core.display.Javascript object>

### Plan of use:
* 2 Passes:
    * 1) correct all typos in words we care about (no phrase tokens allowed)
    * 2) replace words with relevant word.

In [ ]:
PLAN for dealing with PHRASES:
    list_of_words = [i.split() or i in list_of_phrase]
    fixtypos = insitalize(list_of_words)
    replacestuff = insitiliaize(list_of_phrase)
    for i in blah:
        i = fixtypos(i)
        i = replacestuff(i)
        
PLAN for dealing with non-phrases(it's all just words):
        for i in blah:
            i = fixtypos(i, replace=fruit)
        

In [35]:
f = fuzzy_typos.fuzzy_typos(hazardwords)

In [36]:
start = time.time()
for key in nm_ind.text_dict.keys():
    nm_ind.text_dict[key] = f.fix_typos(nm_ind.text_dict[key])
end = time.time()

In [37]:
print('default hazard words time: ' + str(end - start), )


default hazard words time: 2382.689493894577


40 minutes! 

In [ ]:
formula = 
#TODO

In [ ]:

'nutramigen'
 ('neocate', 0.6790235042572021)
 cartons
('powdered_milk', 0.6593323945999146),
infasoy
 ('comfort_milk', 0.6825845837593079), #brand name which people dont write formula alongside

In [ ]:
cereal = 

In [ ]:
baby_rice #this one is really useful / important. idk how exactly to handle it.
 ('rusk', 0.7649476528167725),
 ('rusks', 0.7439418435096741),
 ('weetabix', 0.7281665802001953),
 ('ready_brek', 0.6686376929283142),
 ('rice', 0.6519530415534973),

 ('readybrek', 0.6332829594612122),
cornflakes
muesli
'bran_flakes'
'cheerios'
'shreddies'
'rice_cakes'
'rice_pudding'


 ('babyrice', 0.648690402507782),

In [ ]:
puree_things = 

In [ ]:
mashed


In [ ]:
fruit = \
['mashed_banana',
'berries',
'blueberries',
'raisins',
'apples',
'pear',
'strawberries',
'pineapple', 
'raspberries',
'mango', 
'prunes', 
'fruit_pots']
grapefruit
plum

In [ ]:
veg = \
['mushroom', 
'red_pepper',
'green_beans', 
'courgette', 
'broccoli', 
'tomato',
'parsnips', 
'greens', 
'potato', 
'carrots',
'broccoli',
'cucumber', 
'peas', 
'tomatoes', 
'sweet_potato',
'sweetcorn', 
'corn', 
'spinach', 
'cauliflower',
'butternut_squash', 
'beetroot']
 ('squash', 0.5779417157173157),

In [ ]:
tinned premade canned jarred pouches
 ('ready_made', 0.6789427995681763),
     ('ellas_kitchen', 0.8299736976623535), # only sells baby food. many varieties. replace w "baby food"
    